In [4]:
import pandas as pd
import requests
from census import Census
# Census API Key
from config import api_key

In [12]:
c19 = Census(api_key, year=2019)
# See: https://github.com/datamade/census for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data19 = c.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_19 = pd.DataFrame(census_data19)

# Column Reordering
census_19 = census_19.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_19["PovertyRate"] = 100 * \
    census_19["PovertyCount"].astype(
        int) / census_19["Population"].astype(int)
# Add in Employment Rate (Employment Count / Population)
census_19["UnemploymentRate"] = 100 * \
    census_19["UnemploymentCount"].astype(
        int) / census_19["Population"].astype(int)
# Final DataFrame
census_19 = census_19[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_19

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,01,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,02,Alaska,737068.0,76933.0,10.437707,3.637114,3434.0,108797.0,41109.0,88058.0,476015.0,24205.0,109751.0,45920.0,9204.0,51870.0,60665.0
2,04,Arizona,7050299.0,1043764.0,14.804535,2.778676,71270.0,940168.0,408312.0,869452.0,5444453.0,317462.0,317414.0,233213.0,14458.0,2208663.0,263037.0
3,05,Arkansas,2999370.0,496260.0,16.545475,2.349860,23602.0,559816.0,143979.0,297250.0,2301044.0,459542.0,20434.0,45504.0,8733.0,224130.0,80537.0
4,06,California,39283497.0,5149742.0,13.109174,3.052765,713452.0,4813250.0,2073823.0,5603047.0,23453222.0,2274108.0,303998.0,5692423.0,155290.0,15327688.0,1922664.0
5,08,Colorado,5610349.0,565873.0,10.086235,2.341583,37709.0,667781.0,321903.0,981017.0,4712574.0,233647.0,54847.0,178147.0,8643.0,1208172.0,205724.0
6,10,Delaware,957248.0,109400.0,11.428595,2.766368,6505.0,187323.0,52636.0,124632.0,658237.0,212302.0,3729.0,37009.0,542.0,88364.0,27079.0
7,11,District of Columbia,692683.0,107140.0,15.467393,4.037489,5406.0,70065.0,14792.0,122380.0,285857.0,320811.0,2091.0,27592.0,376.0,76191.0,21445.0
8,09,Connecticut,3575074.0,344146.0,9.626262,3.257443,26484.0,588032.0,191964.0,541380.0,2714031.0,383416.0,9955.0,161257.0,1123.0,574240.0,119000.0
9,12,Florida,20901636.0,2870487.0,13.733313,2.686144,224991.0,3647514.0,1468744.0,2827938.0,15702256.0,3359031.0,59320.0,571276.0,12653.0,5346684.0,572021.0


In [13]:
c18 = Census(api_key, year=2018)
# See: https://github.com/datamade/census for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data18 = c.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_18 = pd.DataFrame(census_data18)

# Column Reordering
census_18 = census_18.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_18["PovertyRate"] = 100 * \
    census_18["PovertyCount"].astype(
        int) / census_18["Population"].astype(int)
# Add in Employment Rate (Employment Count / Population)
census_18["UnemploymentRate"] = 100 * \
    census_18["UnemploymentCount"].astype(
        int) / census_18["Population"].astype(int)
# Final DataFrame
census_18 = census_18[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_18.head()

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,01,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,02,Alaska,737068.0,76933.0,10.437707,3.637114,3434.0,108797.0,41109.0,88058.0,476015.0,24205.0,109751.0,45920.0,9204.0,51870.0,60665.0
2,04,Arizona,7050299.0,1043764.0,14.804535,2.778676,71270.0,940168.0,408312.0,869452.0,5444453.0,317462.0,317414.0,233213.0,14458.0,2208663.0,263037.0
3,05,Arkansas,2999370.0,496260.0,16.545475,2.349860,23602.0,559816.0,143979.0,297250.0,2301044.0,459542.0,20434.0,45504.0,8733.0,224130.0,80537.0
4,06,California,39283497.0,5149742.0,13.109174,3.052765,713452.0,4813250.0,2073823.0,5603047.0,23453222.0,2274108.0,303998.0,5692423.0,155290.0,15327688.0,1922664.0


In [14]:
c17 = Census(api_key, year=2017)
# See: https://github.com/datamade/census for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data17 = c.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_17 = pd.DataFrame(census_data18)

# Column Reordering
census_17 = census_17.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_17["PovertyRate"] = 100 * \
    census_17["PovertyCount"].astype(
        int) / census_17["Population"].astype(int)
# Add in Employment Rate (Employment Count / Population)
census_17["UnemploymentRate"] = 100 * \
    census_17["UnemploymentCount"].astype(
        int) / census_17["Population"].astype(int)
# Final DataFrame
census_17 = census_17[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_17.head()

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,01,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,02,Alaska,737068.0,76933.0,10.437707,3.637114,3434.0,108797.0,41109.0,88058.0,476015.0,24205.0,109751.0,45920.0,9204.0,51870.0,60665.0
2,04,Arizona,7050299.0,1043764.0,14.804535,2.778676,71270.0,940168.0,408312.0,869452.0,5444453.0,317462.0,317414.0,233213.0,14458.0,2208663.0,263037.0
3,05,Arkansas,2999370.0,496260.0,16.545475,2.349860,23602.0,559816.0,143979.0,297250.0,2301044.0,459542.0,20434.0,45504.0,8733.0,224130.0,80537.0
4,06,California,39283497.0,5149742.0,13.109174,3.052765,713452.0,4813250.0,2073823.0,5603047.0,23453222.0,2274108.0,303998.0,5692423.0,155290.0,15327688.0,1922664.0


In [15]:
c16 = Census(api_key, year=2016)
# See: https://github.com/datamade/census for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data16 = c.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_16 = pd.DataFrame(census_data16)

# Column Reordering
census_16 = census_16.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_16["PovertyRate"] = 100 * \
    census_16["PovertyCount"].astype(
        int) / census_16["Population"].astype(int)
# Add in Employment Rate (Employment Count / Population)
census_16["UnemploymentRate"] = 100 * \
    census_16["UnemploymentCount"].astype(
        int) / census_16["Population"].astype(int)
# Final DataFrame
census_16 = census_16[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_16.head()

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,01,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,02,Alaska,737068.0,76933.0,10.437707,3.637114,3434.0,108797.0,41109.0,88058.0,476015.0,24205.0,109751.0,45920.0,9204.0,51870.0,60665.0
2,04,Arizona,7050299.0,1043764.0,14.804535,2.778676,71270.0,940168.0,408312.0,869452.0,5444453.0,317462.0,317414.0,233213.0,14458.0,2208663.0,263037.0
3,05,Arkansas,2999370.0,496260.0,16.545475,2.349860,23602.0,559816.0,143979.0,297250.0,2301044.0,459542.0,20434.0,45504.0,8733.0,224130.0,80537.0
4,06,California,39283497.0,5149742.0,13.109174,3.052765,713452.0,4813250.0,2073823.0,5603047.0,23453222.0,2274108.0,303998.0,5692423.0,155290.0,15327688.0,1922664.0


In [16]:
c15 = Census(api_key, year=2015)
# See: https://github.com/datamade/census for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data15 = c.acs5.get(("NAME", "B01003_001E",
                                      "B17001_002E",
                                      "B23025_005E",
                                      "B15003_002E",
                                      "B15003_017E",
                                      "B15003_021E",
                                      "B15003_022E",
                                      "B02001_002E",
                                      "B02001_003E",
                                      "B02001_004E",
                                      "B02001_005E",
                                      "B02001_006E",
                                      "B03001_003E",
                                      "B02001_008E"), {'for': 'state:*'})

# Convert to DataFrame
census_15 = pd.DataFrame(census_data15)

# Column Reordering
census_15 = census_15.rename(columns={"B01003_001E": "Population",
                                      "B17001_002E": "PovertyCount",
                                      "B23025_005E": "UnemploymentCount",
                                      "B15003_002E": "NoCompletedEducation",
                                      "B15003_017E": "PopulationWithHighSchoolDiploma",
                                      "B15003_021E": "PopulationWithAssociatesDegree",
                                      "B15003_022E": "PopulationWithBachelorsDegree",
                                      "B02001_002E": "WhitePop",
                                      "B02001_003E": "BlackPop",
                                      "B02001_004E": "NativeAmericanPop",
                                      "B02001_005E": "AsianPop",
                                      "B02001_006E": "NativeHawaiianPop",
                                      "B03001_003E": "HispanicPop",
                                      "B02001_008E": "TwoOrMoreRacesPop",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_15["PovertyRate"] = 100 * \
    census_15["PovertyCount"].astype(
        int) / census_15["Population"].astype(int)
# Add in Employment Rate (Employment Count / Population)
census_15["UnemploymentRate"] = 100 * \
    census_15["UnemploymentCount"].astype(
        int) / census_15["Population"].astype(int)
# Final DataFrame
census_15 = census_15[["State", "Name", "Population", "PovertyCount", "PovertyRate", "UnemploymentRate",
                        "NoCompletedEducation", "PopulationWithHighSchoolDiploma",
                      "PopulationWithAssociatesDegree", "PopulationWithBachelorsDegree",
                      "WhitePop", "BlackPop", "NativeAmericanPop", "AsianPop", "NativeHawaiianPop",
                      "HispanicPop", "TwoOrMoreRacesPop"]]

census_15.head()

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,01,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,02,Alaska,737068.0,76933.0,10.437707,3.637114,3434.0,108797.0,41109.0,88058.0,476015.0,24205.0,109751.0,45920.0,9204.0,51870.0,60665.0
2,04,Arizona,7050299.0,1043764.0,14.804535,2.778676,71270.0,940168.0,408312.0,869452.0,5444453.0,317462.0,317414.0,233213.0,14458.0,2208663.0,263037.0
3,05,Arkansas,2999370.0,496260.0,16.545475,2.349860,23602.0,559816.0,143979.0,297250.0,2301044.0,459542.0,20434.0,45504.0,8733.0,224130.0,80537.0
4,06,California,39283497.0,5149742.0,13.109174,3.052765,713452.0,4813250.0,2073823.0,5603047.0,23453222.0,2274108.0,303998.0,5692423.0,155290.0,15327688.0,1922664.0


In [18]:
census_15["Year"] = "2015"
census_16["Year"] = "2016"
census_17["Year"] = "2017"
census_18["Year"] = "2018"
census_19["Year"] = "2019"

In [19]:
census_combined = census_15.append(census_16)
census_combined = census_combined.append(census_17)
census_combined = census_combined.append(census_18)
census_combined = census_combined.append(census_19)
census_combined

,State,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop,Year
0,01,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0,2015
1,02,Alaska,737068.0,76933.0,10.437707,3.637114,3434.0,108797.0,41109.0,88058.0,476015.0,24205.0,109751.0,45920.0,9204.0,51870.0,60665.0,2015
2,04,Arizona,7050299.0,1043764.0,14.804535,2.778676,71270.0,940168.0,408312.0,869452.0,5444453.0,317462.0,317414.0,233213.0,14458.0,2208663.0,263037.0,2015
3,05,Arkansas,2999370.0,496260.0,16.545475,2.349860,23602.0,559816.0,143979.0,297250.0,2301044.0,459542.0,20434.0,45504.0,8733.0,224130.0,80537.0,2015
4,06,California,39283497.0,5149742.0,13.109174,3.052765,713452.0,4813250.0,2073823.0,5603047.0,23453222.0,2274108.0,303998.0,5692423.0,155290.0,15327688.0,1922664.0,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47,53,Washington,7404107.0,785244.0,10.605519,2.530082,59452.0,925707.0,509353.0,1144545.0,5581128.0,281683.0,94449.0,631333.0,49090.0,937579.0,433386.0,2019
48,54,West Virginia,1817305.0,310044.0,17.060647,2.856428,10076.0,432826.0,94488.0,159196.0,1691600.0,66990.0,3667.0,14523.0,419.0,28368.0,32135.0,2019
49,55,Wisconsin,5790716.0,639160.0,11.037668,1.926601,34775.0,1073887.0,429430.0,780824.0,4946961.0,371200.0,51392.0,161841.0,2400.0,394392.0,138572.0,2019
50,56,Wyoming,581024.0,62257.0,10.715048,2.331057,1881.0,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0,2019


In [21]:
infant_mortality = pd.read_csv("data/2019mortality.csv", usecols=["YEAR", "STATE", "RATE", "DEATHS"])
infant_mortality

,YEAR,STATE,RATE,DEATHS
0,2019,AL,7.89,449
1,2019,AK,4.81,48
2,2019,AZ,5.24,429
3,2019,AR,6.9,251
4,2019,CA,4.06,1879
...,...,...,...,...
345,2005,VA,7.47,781
346,2005,WA,5.07,419
347,2005,WV,8.16,170
348,2005,WI,6.54,464


In [27]:
mortality_clean = infant_mortality[infant_mortality.YEAR != 2005]
mortality_clean = mortality_clean[mortality_clean.YEAR != 2014]
mortality_clean

,YEAR,STATE,RATE,DEATHS
0,2019,AL,7.89,449
1,2019,AK,4.81,48
2,2019,AZ,5.24,429
3,2019,AR,6.9,251
4,2019,CA,4.06,1879
...,...,...,...,...
245,2015,VA,5.9,610
246,2015,WA,4.88,434
247,2015,WV,7.12,141
248,2015,WI,5.8,389


In [32]:
# csv from https://worldpopulationreview.com/states/state-abbreviations
states = pd.read_csv("Data/states.csv", usecols=["State", "Code"])
states.rename(columns={"State": "Name", "Code": "STATE"}, inplace=True)
states.head()

,Name,STATE
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [37]:
mortality_state = pd.merge(mortality_clean, states, on="STATE")

mortality_state.rename(columns={"YEAR": "Year", "RATE": "Rate", "DEATHS": "Deaths", "STATE": "State"}, inplace=True)
mortality_state.head()

,Year,State,Rate,Deaths,Name
0,2019,AL,7.89,449,Alabama
1,2018,AL,6.94,401,Alabama
2,2017,AL,7.38,435,Alabama
3,2016,AL,9.03,534,Alabama
4,2015,AL,8.31,496,Alabama


In [43]:
census_combined["Year"] = census_combined["Year"].astype(int)
# census_combined.info()
final_df = pd.merge(mortality_state, census_combined, how="inner", on=["Name", "Year"])
final_df

,Year,State_x,Rate,Deaths,Name,State_y,Population,PovertyCount,PovertyRate,UnemploymentRate,...,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,2019,AL,7.89,449,Alabama,01,4876250.0,795989.0,16.323794,2.708946,...,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,2018,AL,6.94,401,Alabama,01,4876250.0,795989.0,16.323794,2.708946,...,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
2,2017,AL,7.38,435,Alabama,01,4876250.0,795989.0,16.323794,2.708946,...,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
3,2016,AL,9.03,534,Alabama,01,4876250.0,795989.0,16.323794,2.708946,...,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
4,2015,AL,8.31,496,Alabama,01,4876250.0,795989.0,16.323794,2.708946,...,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2019,WY,7,46,Wyoming,56,581024.0,62257.0,10.715048,2.331057,...,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0
246,2018,WY,5.33,35,Wyoming,56,581024.0,62257.0,10.715048,2.331057,...,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0
247,2017,WY,4.64,32,Wyoming,56,581024.0,62257.0,10.715048,2.331057,...,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0
248,2016,WY,5.01,37,Wyoming,56,581024.0,62257.0,10.715048,2.331057,...,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0


In [45]:
final_df.rename(columns={"State_x": "State"}, inplace=True)
final_df.drop(["State_y"], axis=1, inplace=True)
final_df

,Year,State,Rate,Deaths,Name,Population,PovertyCount,PovertyRate,UnemploymentRate,NoCompletedEducation,PopulationWithHighSchoolDiploma,PopulationWithAssociatesDegree,PopulationWithBachelorsDegree,WhitePop,BlackPop,NativeAmericanPop,AsianPop,NativeHawaiianPop,HispanicPop,TwoOrMoreRacesPop
0,2019,AL,7.89,449,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
1,2018,AL,6.94,401,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
2,2017,AL,7.38,435,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
3,2016,AL,9.03,534,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
4,2015,AL,8.31,496,Alabama,4876250.0,795989.0,16.323794,2.708946,40837.0,841716.0,282316.0,529178.0,3320247.0,1299048.0,25565.0,66270.0,2238.0,208626.0,92220.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,2019,WY,7,46,Wyoming,581024.0,62257.0,10.715048,2.331057,1881.0,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0
246,2018,WY,5.33,35,Wyoming,581024.0,62257.0,10.715048,2.331057,1881.0,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0
247,2017,WY,4.64,32,Wyoming,581024.0,62257.0,10.715048,2.331057,1881.0,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0
248,2016,WY,5.01,37,Wyoming,581024.0,62257.0,10.715048,2.331057,1881.0,93507.0,43761.0,68269.0,531304.0,5582.0,14164.0,5025.0,596.0,57341.0,15521.0


In [46]:
final_df.to_csv("Data/infant_mortality.csv", encoding="utf-8", index=False)